In [ ]:
cd examples/text_to_image/

In [ ]:
pip install git+https://github.com/huggingface/diffusers.git

In [ ]:
pip install -U -r requirements.txt


In [ ]:
!accelerate config default

In [ ]:
!huggingface-cli login


In [ ]:
from datasets import load_dataset

# Load the dataset with streaming
dataset = load_dataset('latentcat/animesfw', split='train', streaming=True)

# Iterate through the first 100 rows
first_100_rows = []
for i, row in enumerate(dataset):
    if i < 100:
        first_100_rows.append(row)
    else:
        break

# Check the number of rows collected
print(f"Number of rows collected: {len(first_100_rows)}")

for i, row in enumerate(first_100_rows):
    print(f"Row {i}: {row}")


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# Extract the image from the row
print (first_100_rows[12]['tags'])

image = first_100_rows[12]['image']

# Display the image using matplotlib
plt.figure(figsize=(5,5))
plt.imshow(image)
plt.axis('off')  # Hide axes
plt.show()


In [ ]:

import json
import os
from PIL import Image

# Directory to save images
image_dir = 'images'
os.makedirs(image_dir, exist_ok=True)

# List to hold data
data = []

# Save each image and store its file path
for i, row in enumerate(first_100_rows):
    image = row['image']
    image_path = os.path.join(image_dir, f'image_{i}.jpg')
    image.save(image_path)
    # Add file path and tags to the data list
    data.append({
        'image': image_path,
        'text': row['tags']
    })

# Save the metadata to a JSON file
with open('first_100_rows.json', 'w') as f:
    json.dump(data, f)


In [ ]:
import os

# Set environment variables
os.environ['MODEL_NAME'] = "CompVis/stable-diffusion-v1-4"
os.environ['DATASET_NAME'] = "first_100_rows.json"


!accelerate launch train_text_to_image.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$DATASET_NAME \
  --use_ema \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --mixed_precision="fp16" \
  --max_train_steps=15000 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --lr_scheduler="constant" --lr_warmup_steps=0 \
  --output_dir="sd-pokemon-model"


In [ ]:
! MODEL_NAME="CompVis/stable-diffusion-v1-4"
! export dataset_name="lambdalabs/naruto-blip-captions"

!accelerate launch train_text_to_image.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$dataset_name \
  --use_ema \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --mixed_precision="fp16" \
  --max_train_steps=100 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --lr_scheduler="constant" --lr_warmup_steps=0 \
  --output_dir="sd-pokemon-model"


In [ ]:
import torch
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt
from PIL import Image

# Load the fine-tuned model
model_path = "sd-pokemon-model"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")
pipe.safety_checker = None
# Run inference
prompt = "a man with dark hair and brown eyes mask"
num_images = 1

with torch.autocast("cuda"):
    images = pipe(prompt, num_images_per_prompt=num_images).images

# Display the generated images


for img in images:
    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.axis('off')  # Hide axes
    plt.show()


In [ ]:
import torch
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt
from PIL import Image

# Load the fine-tuned model
model_path = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")
# Run inference
prompt = " man with dark hair and brown eyes mask"
num_images = 1

with torch.autocast("cuda"):
    images = pipe(prompt, num_images_per_prompt=num_images).images

# Display the generated images


for img in images:
    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.axis('off')  # Hide axes
    plt.show()
